# DrugBank Data Curation and Molecular Feature Generation

## Purpose
This notebook performs the **data acquisition, curation, and molecular preprocessing** steps required to construct a clean, model-ready dataset for predicting CYP450-mediated drug metabolism.  

It corresponds to the **ETL and preprocessing stages** of the thesis workflow and serves as the single source of truth for the curated molecular dataset used in downstream modeling and evaluation.

---

## Inputs

1. **DrugBank-derived tabular data**
   - Curated relational tables exported from DrugBank (academic license)

2. **External chemical information for gap-filling purposes**
   - Compound names are used to query the ChemSpider database via the ChemSpiPy API
   - Only applied to compounds with missing SMILES/InChI

3. **Local Python modules**
   - `api_calls.py`: ChemSpider querying and SMILES/InChI retrieval utilities
   - `embedder.py`: RDKit-based molecular fingerprint generation

---

## Outputs
!!!!!!!!!!!!!!!!!!!!!! TBF
**Curated pandas DataFrame**
   - One row per small-molecule compound
   - Includes:
     - Unique compound identifier (DrugBank ID or derived internal ID)
     - Canonical SMILES
     - Standardized InChI
     - List of compound-specific metabolizing CYP450 enzymes


!!!!!!!!!!!!!!!!!!!!!!!!TBF
---

## Reproducibility and Re-running Considerations

- **Deterministic components**
  - RDKit fingerprint generation is fully deterministic given identical inputs
  - Filtering rules and feature extraction parameters are fixed

- **Non-deterministic components**
  - External APIs are used once to complete missing structures. Retrieved SMILES/InChIs are persisted and treated as frozen thereafter.
  - ChemSpider API queries may return different results over time
  - Network failures or database updates may affect retrieved structures

- **Recommended usage**
  - This notebook should be run **once** to generate a finalized curated dataset
  - The resulting dataset should be persisted and treated as immutable
  - Downstream analysis and modeling should not re-trigger external API calls

---

## Position in the Thesis Workflow

This notebook implements:
- Data acquisition
- Data cleaning and curation
- Molecular representation engineering



-------------
## 1) DrugBank data load

This section loads the core DrugBank tables required for the project and performs an initial filtering step to focus exclusively on small-molecule drugs. The ChemSpider client is initialized here for later use in retrieving missing structural information. The DrugBank datasets are read from local CSV files and inspected to identify and isolate compounds classified as *SmallMoleculeDrug*, excluding biologics such as proteins and peptides that are not relevant for CYP450-mediated metabolism.

In [9]:
import pandas as pd
from chemspipy import ChemSpider

cs = ChemSpider("LtPG7M2FaS9fbwJuCpUCD84WI1QDJ09fuBbr0CN9") #ChemSpider API

csvfile_enzyme = pd.read_csv("../DataBase/drug_enzyme_db_all.csv", delimiter= ",", header= 0, encoding="utf-8")

csvfile_smiles = pd.read_csv("../DataBase/structure_smiles_links_all.csv", delimiter= ",", header= 0, encoding="utf-8")

csvfile_drugs = pd.read_csv("../DataBase/drug_links_all.csv", delimiter= ",", header=0, encoding="utf-8")

csvfile_drugs["Drug Type"].value_counts()

#Get all IDs of SmallMolecules to ignore proteins and peptides
small_molecule_ids = csvfile_drugs.loc[
    csvfile_drugs["Drug Type"] == "SmallMoleculeDrug", "DrugBank ID"]


csvfile_drugs.loc[csvfile_drugs["Drug Type"] == "SmallMoleculeDrug"].head(
)

,DrugBank ID,Name,CAS Number,Drug Type,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,PharmGKB ID,...,GenBank ID,DPD ID,RxList Link,Pdrhealth Link,Wikipedia ID,Drugs.com Link,NDC ID,ChemSpider ID,BindingDB ID,TTD ID
5,DB00006,Bivalirudin,128270-60-0,SmallMoleculeDrug,NaN,D03136,16129704.0,46507415.0,59173.0,PA10032,...,NaN,12945,http://www.rxlist.com/cgi/generic/angiomax.htm,NaN,Bivalirudin,http://www.drugs.com/cdi/bivalirudin.html,NaN,10482069.0,50248103.0,DAP000542
13,DB00014,Goserelin,65807-02-5,SmallMoleculeDrug,NaN,D00573,5311128.0,46507336.0,5523.0,PA164747674,...,NaN,11167,http://www.rxlist.com/cgi/generic/goserel.htm,NaN,Goserelin,http://www.drugs.com/cdi/goserelin.html,NaN,4470656.0,NaN,DAP000023
25,DB00027,Gramicidin D,1405-97-6,SmallMoleculeDrug,NaN,D04369,45267103.0,46507412.0,NaN,PA449808,...,NaN,8523,NaN,NaN,Gramicidin,NaN,NaN,24623445.0,NaN,DAP001327
33,DB00035,Desmopressin,16679-58-6,SmallMoleculeDrug,C06944,D00291,NaN,NaN,4450.0,PA449237,...,NaN,2155; 19910; 20181,http://www.rxlist.com/cgi/generic3/desmoprt.htm,NaN,Desmopressin,http://www.drugs.com/cdi/desmopressin.html,NaN,4470602.0,50205308.0,DNC000526
47,DB00050,Cetrorelix,120287-85-6,SmallMoleculeDrug,NaN,D07665,25074887.0,46505494.0,59224.0,PA164764506,...,NaN,13224,http://www.rxlist.com/cgi/generic2/cetrorelix.htm,NaN,Cetrorelix,http://www.drugs.com/cdi/cetrorelix-acetate.html,NaN,10482082.0,50369965.0,DAP000096


### 2) Simplifying and harmonizing CYP names

In the dataset, cytochrome P450 enzymes appear with their full descriptive names (e.g., *"Cytochrome P450 1A2"*).  
To make the data easier to analyze and compare, we created a function `name_to_cypcode` that converts these long names into their standardized short codes (e.g., *"CYP1A2"*).  

This harmonization has several benefits:
- **Consistency**: using a single standard naming convention avoids duplicates or mismatches.  
- **Clarity**: the shorter CYP codes are widely used in scientific literature and databases, making results easier to interpret.  
- **Efficiency**: simplifies merging tables, performing lookups, and visualizing results.  

By applying this function during preprocessing, we ensure that all references to CYP enzymes in the project are uniform and aligned with common pharmacological standards.  


In [10]:
def name_to_cypcode(CypName: str) -> str | None:
    """
    Convert a cytochrome P450 enzyme name into its standard CYP code.

    Example:
        "Cytochrome P450 1A2" -> "CYP1A2"

    Args:
        CypName (str): Full enzyme name in plain text.

    Returns:
        str | None: Standardized CYP code if input contains "Cytochrome P450",
        otherwise None.
    """
    if "Cytochrome P450" in CypName:
        code: str = "CYP" + CypName.split()[2].strip(",")
        return code
    return

print(name_to_cypcode("Cytochrome P450 2C12, female-specific")) #Data base real example

CYP2C12


### 3) Consolidating drug–CYP associations into a single structure

In this step, we iterate through the enzyme dataset and build a dictionary (`drug_CYPs`) where each drug is represented by its **DrugBank ID** as the key and linked to a **list of CYP codes** as the value.  
This approach ensures that every drug appears only once, with all its associated CYP enzymes collected together in a convenient list format, that can be easily updated with extra data from other sources.

In [11]:
drug_CYPs: dict[str, list[str]] = {}
for _, row in csvfile_enzyme.iterrows():
    drug_id = row["DrugBank ID"]
    cyp_code = name_to_cypcode(row["UniProt Name"])
    
    if cyp_code:  # only add valid CYPs
        drug_CYPs.setdefault(drug_id, []).append(cyp_code)


print(drug_CYPs)

{'DB00008': ['CYP1A2'], 'DB00011': ['CYP1A2'], 'DB00018': ['CYP1A2'], 'DB00022': ['CYP1A2', 'CYP2D6', 'CYP2C9'], 'DB00030': ['CYP1A2'], 'DB00033': ['CYP1A2'], 'DB00034': ['CYP1A2'], 'DB00041': ['CYP3A4', 'CYP2E1'], 'DB00046': ['CYP1A2'], 'DB00047': ['CYP1A2'], 'DB00052': ['CYP1A2', 'CYP2C19'], 'DB00060': ['CYP1A2'], 'DB00068': ['CYP1A2'], 'DB00069': ['CYP1A2'], 'DB00071': ['CYP1A2'], 'DB00082': ['CYP3A4', 'CYP4A11'], 'DB00091': ['CYP2C19', 'CYP2D6', 'CYP3A5', 'CYP3A4'], 'DB00104': ['CYP3A4'], 'DB00105': ['CYP1A2'], 'DB00109': ['CYP2C19', 'CYP2E1'], 'DB00118': ['CYP2E1'], 'DB00121': ['CYP1B1'], 'DB00136': ['CYP3A4'], 'DB00152': ['CYP4B1'], 'DB00162': ['CYP26A1'], 'DB00163': ['CYP3A4'], 'DB00165': ['CYP1A1'], 'DB00169': ['CYP2J2', 'CYP3A4', 'CYP1A1', 'CYP2C8'], 'DB00170': ['CYP1A2', 'CYP2A6', 'CYP1B1', 'CYP2B6', 'CYP2C8', 'CYP2C9', 'CYP2C19', 'CYP2D6', 'CYP2E1', 'CYP3A4', 'CYP3A5', 'CYP3A7', 'CYP1A1'], 'DB00176': ['CYP2D6', 'CYP1A2', 'CYP1A1', 'CYP3A4', 'CYP3A4', 'CYP3A43', 'CYP3A5', 'CY

### 4) Create dataframes

In [12]:
df_cyps = pd.DataFrame(
    [(drug, cyps) for drug, cyps in drug_CYPs.items()],
    columns=["DrugBank ID", "CYPs"])

df_cyps = pd.merge(
    df_cyps,
    csvfile_smiles[["DrugBank ID", "Name", "InChI", "SMILES"]],
    on="DrugBank ID",
    how="left"   # keep all drugs from df_cyps, fill missing InChI/SMILES with NaN
)


#Keep only SmallMoleculeDrugs IDs in the df
df_cyps_smd = df_cyps[df_cyps["DrugBank ID"].isin(small_molecule_ids)]
df_cyps_smd.describe()


,DrugBank ID,CYPs,Name,InChI,SMILES
count,1376,1376,1376,1357,1357
unique,1376,580,1376,1357,1357
top,DB16236,[CYP3A4],Mitapivat,InChI=1S/C24H25NO4/c1-25-13-12-24-11-10-18(28-...,[H][C@]12C[C@@H](OC(=O)C3=CC=CC=C3)C=C[C@]11CC...
freq,1,298,1,1,1


### 5. Retrieval and Caching of Missing Molecular Structures


In [13]:
missing_structures = df_cyps_smd.loc[df_cyps_smd["InChI"].isnull()]
missing_structures.head()

,DrugBank ID,CYPs,Name,InChI,SMILES
90,DB00286,"[CYP3A4, CYP1A2]",Conjugated estrogens,NaN,NaN
467,DB01049,[CYP3A4],Ergoloid mesylate,NaN,NaN
753,DB05115,[CYP1A2],NN344,NaN,NaN
769,DB05488,[CYP1A2],Technetium Tc-99m ciprofloxacin,NaN,NaN
1046,DB09317,[CYP3A4],"Synthetic Conjugated Estrogens, A",NaN,NaN


This section completes the structural information for compounds that lack SMILES representations in the original DrugBank dataset. The ChemSpider API is used to query these compounds by name and retrieve canonical SMILES strings, which are then converted into standardized InChI identifiers using RDKit.  

All code in this section is intentionally commented out because this operation is designed to be executed **only once**. The results are cached to disk and reused in subsequent runs to ensure reproducibility, avoid unnecessary external API calls, and prevent exceeding ChemSpider usage limits. Once the missing structures have been retrieved and stored locally, the curated dataset can be treated as frozen and used safely in downstream preprocessing and modeling steps without further dependence on external services.



In [17]:
# This block demonstrates the retrieval of missing SMILES using
# the ChemSpider API. Although functional, it should be run only
# once and the results cached locally to avoid repeated API calls.

import numpy as np
import pandas as pd
import sys
import os
# Add the parent directory (repository root) to sys.path
sys.path.append(os.path.abspath('..'))

from api_calls import get_SMILES_chemspipy, smiles_to_inchi

missing_SMILES = get_SMILES_chemspipy(missing_structures)

# Number of compounds for which at least one SMILES was retrieved
n_records = len(missing_SMILES)

# Number of unique DrugBank IDs affected
n_unique_drugs = len({t.drugbank_id for t in missing_SMILES})

print(f"Number of ChemSpider records retrieved: {n_records}")
print(f"Number of DrugBank compounds with recovered structures: {n_unique_drugs}")

new_smiles_df = pd.DataFrame([
    {
        "DrugBank ID": t.drugbank_id,      
        "CYPs": t.cyps,             
        "Name": t.common_name,
        "InChI": np.nan, 
        "SMILES": t.smiles      
    }
    for t in missing_SMILES])

### Get InChI from SMILES 
new_smiles_df["InChI"] = new_smiles_df["SMILES"].apply(smiles_to_inchi)
### Cache results locally to avoid repeated API usage
# new_smiles_df.to_csv("smiles_data_cache.csv", index=False)

new_smiles_df.head()


Number of ChemSpider records retrieved: 15
Number of DrugBank compounds with recovered structures: 10


[20:42:11] WARNING: Proton(s) added/removed



,DrugBank ID,CYPs,Name,InChI,SMILES
0,DB00286,"[CYP3A4, CYP1A2]","Sodium (8ξ,9ξ,14ξ)-17-oxoestra-1,3,5(10)-trien...",InChI=1S/C18H22O5S.Na/c1-18-9-8-14-13-5-3-12(2...,C[C@]12CCC3c4ccc(OS(=O)(=O)[O-])cc4CCC3C1CCC2=...
1,DB01049,[CYP3A4],Ergoloid mesylate,InChI=1S/C29H37N5O5.CH4O3S/c1-15(2)28(27(37)34...,CC(C)[C@@]1(NC(=O)[C@@H]2C[C@@H]3c4cccc5[nH]cc...
2,DB09568,"[CYP3A4, CYP3A43, CYP3A5, CYP3A7, CYP4F2]",Eicosapentanoic acid,InChI=1S/C20H30O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCC(=O)O
3,DB09568,"[CYP3A4, CYP3A43, CYP3A5, CYP3A7, CYP4F2]",CLUPANODONIC ACID,InChI=1S/C22H34O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCCC(=O)O
4,DB09568,"[CYP3A4, CYP3A43, CYP3A5, CYP3A7, CYP4F2]",Osbond acid,InChI=1S/C22H34O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,CCCCC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O


### 6) Final cleaning and Removal of duplicates

The dataset is cleaned by removing entries without valid SMILES and eliminating duplicate molecules based on their InChI (as they uniquely identify compounds), keeping only the first occurrence to ensure unique and consistent molecular representations.

In [18]:
df_DrugBank = pd.concat([df_cyps_smd, new_smiles_df], ignore_index=True)
df_DrugBank_clean = df_DrugBank.dropna(subset=["SMILES"])

print(f"Number of unique compounds before cleaning: {df_DrugBank_clean['DrugBank ID'].nunique()}")

### InChI duplicates: keep the first occurrence; others are flagged for removal
duplicate_inchi = df_DrugBank_clean[df_DrugBank_clean.duplicated(subset=["InChI"], keep="first")]
### Gather DrugBank IDs of duplicates
ids_to_drop = duplicate_inchi["DrugBank ID"].tolist() 
### Remove duplicates from the main DataFrame
df_DrugBank_clean = df_DrugBank_clean[~df_DrugBank_clean["DrugBank ID"].isin(ids_to_drop)]
df_DrugBank_clean = df_DrugBank_clean[df_DrugBank_clean['DrugBank ID'] != 'DB00515'] 
# We drop cis-platin because MolE fails to process it, likely due to its complex structure with metal coordination. 
# This is a known limitation of many cheminformatics tools when handling organometallic compounds. 


print(f"Number of unique compounds after cleaning: {df_DrugBank_clean['DrugBank ID'].nunique()}")
ids_to_drop

Number of unique compounds before cleaning: 1367
Number of unique compounds after cleaning: 1363


['DB11094', 'DB11251', 'DB13385']

### 7) Count CYPs for Ecploratory Data Analysis

To better understand the distribution of target classes in the dataset, we compute the number of molecules associated with each CYP450 enzyme:

1. **Flattening the CYP lists**:  
   Each molecule may be metabolized by multiple CYP enzymes. Using Python's `Counter`, we aggregate all occurrences across the dataset.

2. **Summary table creation**:  
   The results are stored in `df_cyp_counter`, which lists each CYP enzyme alongside the number of molecules (`DrugCount`) linked to it.

3. **Statistical overview**:  
   Descriptive statistics (`describe()`) provide insight into the distribution of molecules per enzyme, helping identify class imbalances and guiding subsequent preprocessing and model selection steps.

In [19]:
###CYP counter###
from collections import Counter

cyp_counter = Counter(cyp for cyps in df_DrugBank_clean["CYPs"] for cyp in cyps)

df_cyp_counter = pd.DataFrame([(cyp, count) for cyp, count in cyp_counter.items()], 
                            columns= ["CYP", "DrugCount"])
print("CYP count stats before curation:")
df_cyp_counter["DrugCount"].describe()

CYP count stats before curation:


count      33.000000
mean      127.121212
std       215.675937
min         1.000000
25%         2.000000
50%        17.000000
75%       179.000000
max      1051.000000
Name: DrugCount, dtype: float64

In [ ]:
#Eliminate low frequency CYPs
THRESHOLD = 10
uncommon_cyps = [k for k, v in cyp_counter.items() if v < THRESHOLD]
df_DrugBank_curated = df_DrugBank_clean.copy()
df_DrugBank_curated["CYPs"] = df_DrugBank_clean["CYPs"].apply(
    lambda cyps: [cyp for cyp in cyps if cyp not in uncommon_cyps])

cyp_counter_curated = Counter(cyp for cyps in df_DrugBank_curated["CYPs"] for cyp in cyps)
df_cyp_counter_curated = pd.DataFrame([(cyp, count) for cyp, count in cyp_counter_curated.items()], 
                            columns= ["CYP", "DrugCount"])
print("Full CYP count stats after curation:")
df_cyp_counter_curated["DrugCount"].describe()

Full CYP count stats after curation:


count      20.00000
mean      208.20000
std       246.52695
min        11.00000
25%        27.25000
50%       125.50000
75%       335.50000
max      1051.00000
Name: DrugCount, dtype: float64

: 